In [67]:
from langchain.document_loaders import PyPDFLoader
import os

from langchain.document_loaders import UnstructuredExcelLoader
from langchain.vectorstores import Chroma
from config import OPENAI_API_KEY


# loader = PyPDFLoader("../prof.pdf")
loader = UnstructuredExcelLoader(
    "excel/faculty.xlsx"
)


data = loader.load()

In [68]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
all_splits = text_splitter.split_documents(data)

In [69]:
# Store 
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=all_splits,embedding=OpenAIEmbeddings())

In [70]:
question = "Who is Anupam Sobti?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [71]:
print(docs)

[Document(page_content='Dr. Dhiraj Sinha', metadata={'source': '../excel/faculty.xlsx'}), Document(page_content='Dr. Sunita Chauhan', metadata={'source': '../excel/faculty.xlsx'}), Document(page_content='Dr. Amrik Sen', metadata={'source': '../excel/faculty.xlsx'}), Document(page_content='Dr. Anupam Sobti\n      My research interest lies at the intersection of computer vision,\\nmachine learning and embedded systems. Smart sensors, assistive\\ndevices, wearable health monitoring systems, etc. require robust\\nand latency-sensitive sensing and actuation. Therefore, these\\napplications require innovation across sensing techniques,\\nperception algorithms and energy optimization techniques in\\nsoftware and hardware. My current research explores the use of\\ndifferent modalities like radar and ultrasonic sensors along with\\nvision sensors for an energy-efficient and high-performance system.\\nI like to build systems. It is when systems are built and verified with\\nthe users that real p

In [72]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
from langchain.chains import RetrievalQA
question = "Who is Dr. Amrik Sen"
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

{'query': 'Who is Dr. Amrik Sen',
 'result': 'Dr. Amrik Sen is a researcher and educator who obtained his Masters and Ph.D. in Applied Mathematics from the University of Colorado, Boulder, USA. He has research experience at the National Center for Atmospheric Research in Boulder, Colorado. He is currently the principal investigator of two research grants sponsored by the Government of India and is leading a multi-university research project. He has taught mathematics courses at both undergraduate and postgraduate levels in India and America.'}

In [73]:
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
llm = GPT4All(model="/Users/rlm/Desktop/Code/gpt4all/models/nous-hermes-13b.ggmlv3.q4_0.bin",max_tokens=2048)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/uniqueusman/.local/lib/python3.10/site-packages/langchain/llms/gpt4all.py", line 136, in validate_environment
ModuleNotFoundError: No module named 'gpt4all'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/uniqueusman/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_177811/360664772.py", line 3, in <module>
    llm = GPT4All(model="/Users/rlm/Desktop/Code/gpt4all/models/nous-hermes-13b.ggmlv3.q4_0.bin",max_tokens=2048)
  File "/home/uniqueusman/.local/lib/python3.10/site-packages/langchain/load/serializable.py", line 61, in __init__
    """
  File "pydantic/main.py", line 339, in pydantic.main.BaseModel.__init__
  File "pydantic/main.py", line 1102, in pydantic.main.validate_model
  File "/home/uniqueusman/.local/lib/python3.10/site-packages/langcha

In [74]:
question = "Who is Dr. Rucha Joshi"
qa_chain({"query": question})

{'query': 'Who is Dr. Rucha Joshi',
 'result': "I don't have enough information to answer that question."}

In [75]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """You are owned by Plaksha University. You help with connecting user who have a particular interest in a topic to a Professor who have experience in such topic. You also can answer questions regarding a professor.

If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 

You can only get the answer from the context:-
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

result = qa_chain({"query": question})
result["result"]

'Dr. Rucha Joshi is a professor at Plaksha University.'

In [76]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},chain_type="stuff")

In [77]:
result = qa_chain({"query": question})
result["result"]

'Dr. Rucha Joshi is a professor at Plaksha University.'